In [ ]:
import json
from datetime import datetime
from collections import defaultdict

In [ ]:

tracker_file = "result/tracker_memory.json"


File result/tracker_memory.json berhasil dibuat.


In [3]:
# Step 1: Hitung kebutuhan kalori default berdasarkan input user
def calculate_daily_targets(gender, weight_kg, height_cm, age, activity_level="sedentary"):
    """
    Hitung target kalori dan protein harian default berdasarkan BMI dan TDEE.
    Args:
        gender (str): "male" atau "female"
        weight_kg (float): Berat badan dalam kg
        height_cm (float): Tinggi badan dalam cm
        age (int): Usia dalam tahun
        activity_level (str): Tingkat aktivitas ("sedentary", "moderate", "active")

    Returns:
        dict: Target harian default (calories, protein)
    """
    # BMR calculation using Harris-Benedict formula
    if gender.lower() == "male":
        bmr = 10 * weight_kg + 6.25 * height_cm - 5 * age + 5
    else:
        bmr = 10 * weight_kg + 6.25 * height_cm - 5 * age - 161

    # TDEE multiplier based on activity level
    activity_multipliers = {
        "sedentary": 1.2,
        "moderate": 1.55,
        "active": 1.75
    }
    tdee = bmr * activity_multipliers.get(activity_level, 1.2)

    # Protein target (1.2g per kg of body weight as standard)
    protein_target = 1.2 * weight_kg

    return {
        "calories": round(tdee, 2),
        "protein": round(protein_target, 2)
    }

In [6]:
# Step 2: Masukkan input makanan dari user
def add_food_input(food_name, calories, protein, fat, carbs):
    """
    Menambahkan input makanan ke tracker memory.
    Args:
        food_name (str): Nama makanan
        calories (float): Jumlah kalori
        protein (float): Jumlah protein
        fat (float): Jumlah lemak
        carbs (float): Jumlah karbohidrat
    """
    # Waktu input
    now = datetime.now()
    date_str = now.strftime("%Y-%m-%d")  # Hanya tanggal
    time_str = now.strftime("%H:%M:%S")  # Jam, menit, detik

    # Buat data makanan
    food_data = {
        "food_name": food_name,
        "calories": calories,
        "protein": protein,
        "fat": fat,
        "carbs": carbs,
        "time": time_str
    }
 # Load tracker memory
    try:
        with open(tracker_file, "r") as file:
            tracker_memory = json.load(file)
    except FileNotFoundError:
        tracker_memory = defaultdict(list)

    # Tambahkan data ke tanggal yang sesuai
    tracker_memory[date_str].append(food_data)

    # Simpan kembali ke file
    with open(tracker_file, "w") as file:
        json.dump(tracker_memory, file, indent=4)

    print(f"Data untuk {food_name} berhasil ditambahkan.")

In [7]:
# Step 3: Akumulasi total harian
def calculate_daily_totals(date):
    """
    Hitung total kalori dan nutrisi pada tanggal tertentu.
    Args:
        date (str): Tanggal dalam format YYYY-MM-DD

    Returns:
        dict: Total nutrisi harian (calories, protein, fat, carbs)
    """
    try:
        with open(tracker_file, "r") as file:
            tracker_memory = json.load(file)
    except FileNotFoundError:
        print("Tracker memory kosong.")
        return {}

    # Ambil data untuk tanggal yang sesuai
    daily_data = tracker_memory.get(date, [])
    totals = {
        "calories": 0.0,
        "protein": 0.0,
        "fat": 0.0,
        "carbs": 0.0
    }

    for food in daily_data:
        totals["calories"] += food["calories"]
        totals["protein"] += food["protein"]
        totals["fat"] += food["fat"]
        totals["carbs"] += food["carbs"]

    return {key: round(value, 2) for key, value in totals.items()}
